# Compound Eye Ommatidia Analysis
This notebook will grab data from either HDF5 or a TIFF stack, and find the diameter of all the ommatidia, as well as interommatidial angle between all nearest neighbors. IT WILL DO THIS FOR ALL LABELED DATA. This means bad cones, artifacts, anything. In theory I could add some data cleaning stuff, but ideally that is all done before using this. I have mainly tested this on chunks of data (instead of an entire eye), with the goal of combining the data in post. It _should_ work for full eye sets, but I have not tested. Email (griffinb@uchicago.edu) or slack me or whatever if you have questions, suggestions, or want to see code for this being run on one cone with PCA and process visualization - it's quite cool. 

This also has a simple process for saving output so input/PCA only has to be run once on a dataset, and then reloading the output of that is almost instantaneous. It also can take data from the output of Prof. Palmer's MATLAB code. 

I also use a stupid number of Jupyter Lab extensions for this. If you have issues let me know and I can make/send a non-extension one (which will break my heart). 


#### Other assumptions/notes:
- Background is zero
- Most units have been converted to micrometers (which really only means diameters)
- Because I'm dumb, much of this code is intended to be run in order! 

#### #To-Do:
- Circles around highest volume area?
- Implement [COO](https://sparse.pydata.org/en/stable/#)

#### Full list of Jupyter Lab extensions I used:
- [jupyter-matplotlib](https://github.com/matplotlib/ipympl)
- [jupyterlab-manager](https://github.com/jupyter-widgets/ipywidgets)

#### Trouble shooting
- If Imports fail, remove the last line
- If you're getting errors about tqdm, remove it (e.g. go from notebook.tqdm(\_\_\_\_\_\_) to just \_\_\_\_\_\_). This will not remove any functionality.

# Part 0: Imports
If the import cell below fails, try commenting out the last line (%matplotlib qt)

In [1]:
import h5py
import numpy as np
from tqdm import notebook
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from skimage.transform import rescale,resize
from skimage import morphology
import math 
import os 
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.neighbors import NearestNeighbors
from collections import defaultdict
import pickle
import scipy.io as sio
import json
import csv

%matplotlib qt

# Part 1: Let's get our data

Write the resolution (nm^3 per voxel) of your dataset here

In [2]:
px_to_nm = 600

## Option A: New dataset
Run this if you have a dataset that you have not run this code with before. 

It should be either an HDF5 or a TIFF sequence. If you want other implementations, let me know, it's super easy.

**Note:** This may take a while (2+ hours on some large datasets). If you have any idea how to make this more efficient, please let me know I would love to implement it.

Write RELATIVE location + title of HDF5 (e.g. /Griffin/Cones/ygw15.h5) or location of TIFF stack

In [3]:
f_name = './DONE_HDFS/ygw34/ygw34_done.h5'

In [4]:
if f_name[-3:-1]=='.h':
    f = h5py.File(f_name, 'r')
    data = np.asarray(f['label'], dtype='uint16')
    f.close()
else:
    data = []
    for fname in notebook.tqdm(os.listdir(f_name)):
        im = Image.open(os.path.join(f_name, fname))
        imarray = np.array(im)
        data.append(imarray)

if(np.max(data)==1):
    data=morphology.label(data)

In [5]:
data=np.asarray(data)
pc_dict = defaultdict(list)
shape = data.shape
for x in notebook.tqdm(np.arange(shape[0])):
    for y in np.arange(shape[1]):
        for z in np.arange(shape[2]): 
            if data[x,y,z]!=0:
                pc_dict[data[x,y,z]].append([x,y,z]) 

  0%|          | 0/800 [00:00<?, ?it/s]

In [6]:
orig_data_size = data.shape
print("Shape of full dataset: " + str(orig_data_size))

print("Number of unique labels: " + str(len(pc_dict.keys())))

Shape of full dataset: (800, 868, 828)
Number of unique labels: 908


## Option B: Pre-run data
If you have already run this code on a dataset, and have the Pickle's from that run stored somewhere, you can use this!

This is currently set up to pull stuff out of a Pickle that stores EVERYTHING from a PCA run, and from there you can ignore stuff if you want to. Code to convert a post-PCA run of data is at the bottom of Part 2 of this code.

In [9]:
try: 
    pc_dict.items()
    raise NameError('WARNING: YOU SEEM TO HAVE ALREADY RAN A NEW DATASET THROUGH. RUNNING THIS NEXT SECTION (OPTION B) MAY OVERWRITE.')
except:
    print('')

In [4]:
all_fname = './PRERUN_DATA/ygw33_all_01.pickle'
with open(all_fname, 'rb') as handle:
    [pc_dict, all_diams, all_PCAs, all_centroids, all_first_PCs, pc_norm_dict] = pickle.load(handle)

## Option C: A .mat file output of Prof. Palmer's code

This assumes the output is in exactly the format that was sent to me, which seems kind of likely tbh.

In [94]:
if all_PCAs:
    raise NameError('WARNING: YOU SEEM TO HAVE ALREADY IMPORTED FROM A PRE-PICKLED DATA SET. RUNNING THE NEXT BLOCK (OPTION C) MAY OVERWRITE YOUR DATA')

NameError: WARNING: YOU SEEM TO HAVE ALREADY IMPORTED FROM A PRE-PICKLED DATA SET. RUNNING THE NEXT BLOCK (OPTION C) MAY OVERWRITE YOUR DATA

In [ ]:
f_name = './ygw32cc_01_tester/from_matlab/ygw32cc_01labels_1.mat'
all_mat = sio.loadmat(f_name)

pc_dict = defaultdict(list)

x_pc = all_mat['labels_xvals']
y_pc = all_mat['labels_yvals']
z_pc = all_mat['labels_zvals']

for i in np.arange(1, x_pc.shape[1]+1):
    total = np.transpose([x_pc[0,i-1], y_pc[0,i-1], z_pc[0,i-1]])[0,:,:]
    pc_dict[i] = total

# Part 2: PCA and Diameters

## *No need to run this if you used option B from above and have not made any changes to the PCA analysis done here.*

if all_PCAs:
    throw### Tell me here what 1 pixel is equal to in nanometers.
(e.g. if 1 pixel = 600 nm^3, write 600 here)

In [5]:
if all_PCAs:
    raise NameError('WARNING: YOU ARE READING FROM A PRE-RUN DATA COLLECTION. There is likely no need to run the next block of code, which does PCA.')

In [7]:
all_PCAs = defaultdict(list)
all_diams = defaultdict(list)
all_centroids = defaultdict(list)
all_first_PCs = defaultdict(list)
bad_label = []
all_norm_locs = defaultdict(list)
pc_norm_dict = defaultdict(list)
global_pointcloud = defaultdict(list)
sample_width = 1
# pragma omp parallel for
for item in notebook.tqdm(pc_dict.keys()):
    point_cloud = []
    test_coord = pc_dict[item]                               # Gets all the locations for the current label
    norm_coord = test_coord - np.mean(test_coord, axis=0)    # Normalizes all the coordinates so the centroid is at 0,0,0. Key step for this PCA hack
    
    try:
        pc_norm_dict[item] = (norm_coord) 
        sparse_norm_coord = norm_coord[::sample_width,:]     # This samples every sample_width locations. This makes this process IMPERFECT but FASTER. Lower sample_width will be more precise, higher will be faster
        pca = PCA(n_components=3)                            # Don't do less than 3, and there's really no reason to do more than 3        
        pca.fit(sparse_norm_coord)
        
        [uall,vall,wall]=pca.components_
        
        for i in norm_coord:
            i = i - np.dot(uall,i)*uall
            point_cloud.append(i) 
      
    
        flat_rep = []
        
        for i in point_cloud:
            (x,y) = (np.dot(vall,i), np.dot(wall,i))        # Honestly don't remember what this does. Wrote it in a fevered rush inspired by Philz's Tesora blend.
            flat_rep.append((x,y))
            
        flat_rep=np.asarray(flat_rep)
       
        dist_list = np.sort(list(
            map(lambda x: 
            np.linalg.norm((0,0) - x) , flat_rep)))         # Determines distance for all points in flattened cone)
        
        ## PAST HERE IS A HEURISTIC FOR DETERMINING A REASONABLE RADIUS INTERPRETATION. ##
        ## The math is basically that 2/(.5*r) of the points in a circle are along the circumference,
        ## so if I take the sorted list of distances from center, take that many from the high end, 
        ## and average those values, I should get a pretty good estimate of the diameter of the circle.
        longest_r = 2 * np.max(dist_list)
        ratio = 2/(.5*longest_r)
        num_to_keep = math.floor(ratio*len(dist_list))         # Number of points used to estimate radius.   
        edge_locs = dist_list[len(dist_list)-num_to_keep:]
        
        if(2*np.average(edge_locs))>10:
            ## And below here I just save some of these results to dictionaries
            all_diams[item].append(px_to_nm*2*np.average(edge_locs))
            all_PCAs[item].append(pca.components_)
            all_centroids[item].append(np.mean(test_coord, axis=0))
            all_first_PCs[item].append(uall)
            all_norm_locs[item].append(norm_coord[:])
            test_coord = np.asarray(test_coord)
            global_pointcloud[item].append(test_coord)
        else:
#             print("This is too small: " + str(item))
            bad_label.append(item)
                
    ## This will usually happen if there are less then 2*sample_width(?) points in the label. Helpful to get rid of artifacts and shouldn't happen to pristine data. ##
    except:
#         print("BREAKS ON THIS LABEL: " + str(item))
        bad_label.append(item)

  0%|          | 0/908 [00:00<?, ?it/s]

## This code will save EVERYTHING from PCA above into a single [Pickle file](https://docs.python.org/3/library/pickle.html)! Please run so life is enjoyable!

Also very useful if you want to upload and combine multiple datasets, but I'm not going to implement that here.

In [8]:
path = './PRERUN_DATA/' 
if not os.path.exists(path):
    os.makedirs(path)


all_fname = '/ygw34_all_02'

In [9]:
with open(path+all_fname+'.pickle', 'wb') as handle:
    pickle.dump([pc_dict, all_diams, all_PCAs,
                 all_centroids, all_first_PCs, pc_norm_dict], 
                 handle, protocol=pickle.HIGHEST_PROTOCOL)

# Part 3: Let's look at some *raw* data!

You will notice that I got kind of bored here and started messing around with some other stastics options that may be interesting to consider.

In [10]:
[limx, limy, limz] = [np.max(np.asarray(list(all_centroids.values()))[:,:,0]),
                      np.max(np.asarray(list(all_centroids.values()))[:,:,1]),
                      np.max(np.asarray(list(all_centroids.values()))[:,:,2])]

limtot = np.ceil(np.max((limx,limy,limz)))

## If you want every plot to be equal on every axis, uncomment below this line ##

# limx = limtot
# limy = limtot
# limz = limtot

## Before we really begin, a sanity check: all the centroids in 3D space

In [11]:
fig = plt.figure()
ax = fig.gca(projection='3d')
all_XS = np.asarray(list(all_centroids.values()))[:,:,0] 
all_YS = np.asarray(list(all_centroids.values()))[:,:,1]
all_ZS = np.asarray(list(all_centroids.values()))[:,:,2]
ax.set_title('Centroids in 3D Space')
ax.set_xlim(0,limx)
ax.set_ylim(0,limy)
ax.set_zlim(0,limz)
ax.scatter(all_XS,all_YS,all_ZS)

## Now, a histogram of _all_ the diameters.

In [12]:
fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
ax.hist(np.asarray(list(all_diams.values())), bins=200)
ax.set_title('Histogram of all diameters (nm)')

all_median_diam = np.median(list(all_diams.values()))
print('Average diameter: ' + str(all_median_diam))

Average diameter: 8853.991602766913


### Now we look at how the diameters differ across the specimen

In [13]:
fig = plt.figure()
ax = fig.gca(projection='3d')

diams_arr = np.asarray(list(all_diams.values()))[:,0]
colors = cm.jet(diams_arr/np.max(diams_arr))
colmap = cm.ScalarMappable(cmap=cm.jet)
colmap.set_array(diams_arr)
cb = fig.colorbar(colmap)

ax.set_title('Centroids in 3D Space Heatmapped to Diameter')
ax.scatter(all_XS,all_YS,all_ZS,
           c=colors)
fig.show()
print(all_XS.shape)

(869, 1)


## Now we have a 3D plot of all of the first PCs in 3D space

(e.g. a PC of <.5,.2,.3> will be plotted in 3D at (.5, .2, .3))

This isn't really useful, but it's a good sanity check. For a full data set, this should look like either a full sphere or a half sphere.

In [14]:
fig = plt.figure()
ax = fig.gca(projection='3d')
all_PCAs_arr = np.asarray(list(all_PCAs.values()))
all_first_PCs_arr = np.asarray(list(all_first_PCs.values()))

all_firstP_XS = all_first_PCs_arr[:,:,0]
all_firstP_YS = all_first_PCs_arr[:,:,1]
all_firstP_ZS = all_first_PCs_arr[:,:,2]
ax.set_title('First PCs in 3D Space')
ax.scatter(all_firstP_XS,all_firstP_YS,all_firstP_ZS)

## Now we plot centroids with their attached PCs

Also a sanity check

In [15]:
all_firstP_XS = all_first_PCs_arr[:,:,1]
all_firstP_YS = all_first_PCs_arr[:,:,1]
all_firstP_ZS = all_first_PCs_arr[:,:,2]

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlim(0, limtot)
ax.set_ylim(0, limtot)
ax.set_zlim(0, limtot)
ax.scatter(all_XS, all_YS, all_ZS, alpha=1, linewidth=3)
ax.set_title('Centroids in 3D Space w/ attached first PCs')
ax.quiver(all_XS, all_YS, all_ZS, all_firstP_XS*1000, all_firstP_YS*1000, all_firstP_ZS*1000, color='red', alpha=.2, linewidth=.5)

In [17]:
all_2P_XS = all_PCAs_arr[:,:,0,0]
all_2P_YS = all_PCAs_arr[:,:,0,1]
all_2P_ZS = all_PCAs_arr[:,:,0,2]

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlim(0, limtot)
ax.set_ylim(0, limtot)
ax.set_zlim(0, limtot)
ax.scatter(all_XS, all_YS, all_ZS, alpha=1, linewidth=3)
ax.set_title('Centroids in 3D Space w/ attached first PCs')
ax.quiver(all_XS, all_YS, all_ZS, all_2P_XS*600, all_2P_YS*600, all_2P_ZS*600, color='red', alpha=.2, linewidth=.5)

# Part 3.2: Remove Bad Data (based on diameter)

The most consistent global property of the ommatidia is their diameter, of which they are incredibly consistent. This next chunk of code will remove any data that has a diameter that is more than 1.25x or less than .75x the median diameter of the full dataset.

In [18]:
bad_vals = []

diam_std = np.std(np.asarray(list(all_diams.values())))
bottom = all_median_diam - diam_std
top = all_median_diam + diam_std


for i in all_diams.keys():
    if all_diams.get(i)<bottom or all_diams.get(i)>top:
        bad_vals.append(i)
        
bad_vals=np.asarray(bad_vals)
print("There were " + str(bad_vals.shape[0]) + " bad data.")

There were 111 bad data.


Now we show the centroids of those bad data

In [19]:
fig = plt.figure()
ax = fig.gca(projection='3d')

# fig2 = plt.figure()
# ax2 = fig2.gca(projection='3d')
good_centroids_dict = all_centroids.copy()
good_diams_dict = all_diams.copy()
good_PCAs_dict = all_PCAs.copy()
good_PC_dict = pc_dict.copy()
print(np.asarray(list(pc_dict.keys())).shape)
good_first_PCs_dict = all_first_PCs.copy()
good_pc_norm_dict = pc_norm_dict.copy()

bad_centroids = np.asarray([good_centroids_dict[x] for x in bad_vals])
bad_XS = bad_centroids[:,:,0]
bad_YS = bad_centroids[:,:,1]
bad_ZS = bad_centroids[:,:,2]

for i in bad_vals:
    [curr_x, curr_y, curr_z] = np.asarray(good_centroids_dict.pop(i))[0]
    curr_diam = np.asarray(good_diams_dict.pop(i))
    good_PCAs_dict.pop(i)
    good_PC_dict.pop(i)
    good_first_PCs_dict.pop(i)
    good_pc_norm_dict.pop(i)

ax.set_title('Centroids of Bad Data in 3D')
ax.scatter(bad_XS,bad_YS,bad_ZS)

(908,)


In [20]:
path = './PRERUN_DATA/' 
if not os.path.exists(path):
    os.makedirs(path)
print(len(list(good_centroids_dict.values())))
all_fname = '/ygw18_volumes.pickle'
good_vols = [len(x) for x in list(good_PC_dict.values())]
print(len(good_vols))
with open(path+all_fname, 'wb') as handle:
    pickle.dump(good_vols, 
                 handle, protocol=pickle.HIGHEST_PROTOCOL)

758
797


In [21]:
good_vols_arr = np.asarray(good_vols)

In [23]:
path = './PRERUN_DATA/' 
if not os.path.exists(path):
    os.makedirs(path)


all_fname = '/ygw33_good_data2.pickle'

with open(path+all_fname, 'wb') as handle:
    pickle.dump([good_PC_dict, good_diams_dict, good_PCAs_dict,
                 good_centroids_dict, good_first_PCs_dict, good_pc_norm_dict], 
                 handle, protocol=pickle.HIGHEST_PROTOCOL)

# Part 4: Interommatidial angle (and distance) between nearest neighbors

In which I abuse sklearn.NearestNeighbors

In [24]:
good_PCAs_arr = np.asarray(list(good_PCAs_dict.values()))[:]
good_centroids_arr =  np.asarray(list(good_centroids_dict.values()))[:]
good_diams_arr =  np.asarray(list(good_diams_dict.values()))[:]

In [25]:
nbrs = NearestNeighbors(n_neighbors = 2, 
                        algorithm = 'brute', metric='euclidean', 
                        leaf_size=60, p=5, n_jobs=-1)


nbrs_f=nbrs.fit(good_centroids_arr[:,0,:])
dists, indecs = nbrs_f.kneighbors(good_centroids_arr[:,0,:])
dists=dists*px_to_nm

## First, we look at a histogram of nearest neighbor interommatidial distances.

Uhhh... if someone knows how to use matplotlib better than me and can zoom in on this before loading the graph, this data is kinda cool but not super useful. 

In [26]:
fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
x_range = np.arange(0,np.max(dists),.05)
ax.hist(dists[:,1], bins=100)
ax.set_title('Histogram of Interommatidial Distance Between Nearest Neighbors (nm)')
fig.show()
print('Avg. distance between nearest neighbors: ' + str(np.median(dists[:,1])))

Avg. distance between nearest neighbors: 11139.28527116133


## Next we look at the diameter graph, but this time of GOOD diameters, again because it's super pertinent info right now.

In [27]:
fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
ax.hist(good_diams_arr, bins=200)
ax.set_title('Histogram of GOOD diameters (nm)')

Text(0.5, 1.0, 'Histogram of GOOD diameters (nm)')

## This plots all the centroids and connects the nearest neighbors. This was a really important sanity check honestly.

In [28]:
all_keys = list(good_centroids_dict.keys())

key_indecs = []
for i in indecs:
    k1 = all_keys[i[0]]
    k2 = all_keys[i[1]]
    key_indecs.append((k1,k2))

In [29]:
GXS = good_centroids_arr[:,0,0]
GYS = good_centroids_arr[:,0,1]
GZS = good_centroids_arr[:,0,2]


fig = plt.figure()
ax = fig.gca(projection='3d')

ax.set_title('Connected Neighbors in 3D')
ax.scatter(GXS, GYS, GZS)
for i in key_indecs:
    p1 = good_centroids_dict[i[0]][0]
    p2 = good_centroids_dict[i[1]][0]
    ax.plot((p1[0],p2[0]), (p1[1], p2[1]), (p1[2],p2[2]))
    
lim = np.max([limx, limy, limz])
ax.set_xlim(0, lim)
ax.set_ylim(0, lim)
ax.set_zlim(0, lim)

(0.0, 820.0340471607315)

## Now we calculate interommatidial angle.

Here, I treat the first PC as the angle of the cone. I am equivalently confident that this is appropriate as I am that the PCA hack works above for finding diameter (this is because they are equivalent problems, I think). 

In [30]:
import scipy
all_angles = defaultdict(float)
for i in key_indecs:
    PC1 = np.asarray(all_first_PCs.get(i[0]))[0,:]
    PC2 = np.asarray(all_first_PCs.get(i[1]))[0,:]
    angle = np.arccos(
        np.dot(PC1,PC2) /
        (np.linalg.norm(PC1)*np.linalg.norm(PC2)))
#   all_angles[i[0]] = angle
    all_angles[i[0]] = math.degrees(angle)%90

fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
ax.hist(all_angles.values(), bins=100)
ax.set_title('Histogram of Interommatidial Angle Between Nearest Neighbors (Phi) (Degrees)')
fig.show()
print('Avg. interommatidial angle: ' + str(np.median(np.asarray(list(all_angles.values())))))

Avg. interommatidial angle: 3.4991269049106686


In [31]:
fig = plt.figure()
ax = fig.gca(projection='3d')

angle_arr = np.asarray(list(all_angles.values()))
colors = cm.jet(angle_arr/np.max(angle_arr))
colmap = cm.ScalarMappable(cmap=cm.jet)
colmap.set_array(angle_arr)
cb = fig.colorbar(colmap)

ax.set_title('Interommatidial Angles in 3D Space')
ax.scatter(GXS,GYS,GZS,
           c=colors)
fig.show()

In [32]:
print(good_vols_arr.shape)
print(GXS.shape)

(797,)
(758,)


In [33]:
fig = plt.figure()
ax = fig.gca(projection='3d')

vols_arr = np.asarray(good_vols)
colors = cm.jet(vols_arr/np.max(vols_arr))
colmap = cm.ScalarMappable(cmap=cm.jet)
colmap.set_array(vols_arr)
cb = fig.colorbar(colmap)

ax.set_title('Volume of Ommatidia in 3D Space')
ax.scatter(GXS,GYS,GZS,
           c=colors)
fig.show()

ValueError: 'c' argument has 797 elements, which is inconsistent with 'x' and 'y' with size 758.

# Part 4.2: BARLOW RATIO

This is an attempt to recreate exactly the ratio Barlow wanted (Φ : θ = interommatidial angle / max resolving angle).

I assume wavelength = 500 nm

In [42]:
all_res = []
all_barlow_ratios = []

for i in key_indecs:
    diam = good_diams_dict.get(i[0])[0]
    rayleigh = 1.22 * (5*pow(10,-5)) / (diam  * pow(10, -7)) ##CHECK HERE
    rayleigh = math.degrees(rayleigh)   
    barlow_ratio = all_angles.get(i[1])/rayleigh
    all_res.append(rayleigh)
    all_barlow_ratios.append(barlow_ratio)

all_res = np.asarray(all_res)
all_barlow_ratios = np.asarray(all_barlow_ratios)
print('Avg. resolution angle (deg): ' + str(np.median(all_res)))
fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
x_range = np.arange(0,np.max(dists),.05)
ax.hist(all_res, bins=200)
ax.set_title('Histogram of Theta (Degrees)')
fig.show()

Avg. resolution angle (deg): 4.005366218790115


In [35]:
fig = plt.figure()
ax = fig.gca(projection='3d')

angle_arr = np.asarray(list(all_angles.values()))
colors = cm.jet(all_res/np.max(all_res))
colmap = cm.ScalarMappable(cmap=cm.jet)
colmap.set_array(all_res)
cb = fig.colorbar(colmap)

ax.set_title('Rayleigh Angles in 3D Space')
ax.scatter(GXS,GYS,GZS,
           c=colors)
fig.show()

In [36]:
fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
x_range = np.arange(0,np.max(dists),.05)
ax.hist(all_barlow_ratios, bins=200)
ax.set_title('All Barlow Ratios')
fig.show()
print("Average Barlow Ratio: "+str(np.median(all_barlow_ratios)))

Average Barlow Ratio: 0.9147524864949292


In [37]:
fig = plt.figure()
ax = fig.gca(projection='3d')

barlows_arr = np.asarray(all_barlow_ratios)
colors = cm.jet(barlows_arr/100)
colmap = cm.ScalarMappable(cmap=cm.jet)
colmap.set_array(barlows_arr)
cb = fig.colorbar(colmap)

ax.set_title('Barlow Ratios in 3D Space')
ax.scatter(GXS,GYS,GZS,
           c=colors)
fig.show()

## This will save all of the import data to CSVs

In [39]:
os.getcwd()

'C:\\Users\\griff\\Box\\College\\Honors\\BERTT'

In [41]:
np.savetxt("./DONE_HDFS/ygw33/all_angles2.csv", np.asarray(list(all_angles.values())), delimiter = ',')
np.savetxt("./DONE_HDFS/ygw33/all_rayleighs2.csv", all_res, delimiter=',')
np.savetxt("./DONE_HDFS/ygw33/all_barlows2.csv", all_barlow_ratios, delimiter=',')
np.savetxt("./DONE_HDFS/ygw33/all_diams2.csv", np.asarray(list(good_diams_dict.values())), delimiter=',')
np.savetxt("./DONE_HDFS/ygw33/all_dists2.csv", dists[:,1], delimiter=',')

# Part 5: Average Cone
The argument here is that if we are to simulate light through the cones, the best way to do it is not with one or two selected cones recreated by hand, but a platonic cone created by a sort of mode-like analysis of the other cones. I here take normalized (meaning the centroid is at 0,0,0) point cloud for each sample, rotate them so they are facing the same direction, then heatmap the different points (by heatmap I mean attach to each point a number value saying how often that point shows up in all of the normalized point clouds). This then plots the points that are above a certain occurrence value and saves the pointcloud above that threshold into a 3D HDF5. Also has saving and loading functionality b/c the process of heatmapping all the points can take a while (in theory I may try to speed this up but you also should only be running it once per dataset so like... eh?). 

In [89]:
# fig = plt.figure()
# ax = fig.gca(projection='3d')
full_norm_dict = defaultdict(list)
full_norm_list = []
for i in good_pc_norm_dict.keys():
# for i in np.arange(600,610):
   
    all_vecs = np.asarray(good_PCAs_dict[i])
    if np.size(all_vecs) > 1:

        og_pc = np.transpose(np.asarray(good_pc_norm_dict[i]))
        inv_PCs =np.linalg.inv(np.transpose(all_vecs[0,:,:]))
        new_pc = np.floor(np.matmul(inv_PCs,og_pc))
        full_norm_dict[i]=np.transpose(new_pc)
        np.append(full_norm_list, np.transpose(new_pc))
#         fig = plt.figure()
#         ax = fig.gca(projection='3d')
#         ax.set_xlim(-25,25)
#         ax.set_ylim(-25,25)
#         ax.set_zlim(-25,25)
#         ax.scatter(new_pc[0,:], new_pc[1,:], new_pc[2,:])
        
full_norm_list=np.asarray(full_norm_list)

In [90]:
x_range = [np.min(new_pc[:,0]), np.max(new_pc[:,0])]
y_range = [np.min(new_pc[:,1]), np.max(new_pc[:,1])]
z_range = [np.min(new_pc[:,2]), np.max(new_pc[:,2])]

loc_heat_map = defaultdict(uint16)
                
for i in notebook.tqdm(full_norm_dict.keys()):
    comp = full_norm_dict[i]
    for loc in comp:
        loc_heat_map[str(loc)]+=1


  0%|          | 0/1061 [00:00<?, ?it/s]

In [91]:
lochm_fname = './DONE_HDFS/ygw18/heatmap_all.pickle'

In [92]:
with open(lochm_fname, 'wb') as handle:
    pickle.dump(loc_heat_map,handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open(lochm_fname, 'rb') as handle:
    loc_heat_map=pickle.load(handle)
    
print(len(loc_heat_map.keys()))

54383


In [97]:
n_loc_heat = loc_heat_map.copy()
plot_locs = []
all_vals = np.asarray(list(n_loc_heat.values()))
print(all_vals.shape)
coloro = 0
limit = 250

for i in loc_heat_map.keys():
    if all_vals[coloro]>limit:
        chunks = i[1:-1].strip().split('.')
        try:
            plot_locs.append([int(chunks[0]),int(chunks[1]),int(chunks[2])])
        except:
            plot_locs.append([0,0,0])
    else:
        n_loc_heat.pop(i)
        all_vals[coloro]=0
    coloro+=1

maxxer = np.max(all_vals)
    
all_vals = np.asarray(list(n_loc_heat.values()))

fig = plt.figure()
ax = fig.gca(projection='3d')
colors = cm.jet(all_vals/np.max(all_vals))
colmap = cm.ScalarMappable(cmap=cm.jet)
colmap.set_array(all_vals)
cb = fig.colorbar(colmap)
ax.set_xlim(-25,25)
ax.set_ylim(-25,25)
ax.set_zlim(-25,25)
plot_locs=np.asarray(plot_locs)
ax.set_title('Points that occur in '+str(math.floor(100*(1-(limit/maxxer)))) +'% of the objects')
ax.scatter(plot_locs[:,0], plot_locs[:,1], plot_locs[:,2], c=colors)
print(plot_locs.shape)

(100106,)
(10982, 3)


In [94]:
x_min = np.min(plot_locs[:,0])
y_min = np.min(plot_locs[:,1])
z_min = np.min(plot_locs[:,2])

trans_locs = plot_locs

trans_locs[:,0]-=x_min
trans_locs[:,1]-=y_min
trans_locs[:,2]-=z_min

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlim(0,50)
ax.set_ylim(0,50)
ax.set_zlim(0,50)
ax.set_title('Points that occur '+str(limit)+' times or more')
ax.scatter(trans_locs[:,0], trans_locs[:,1], trans_locs[:,2])
print(plot_locs.shape)

(16189, 3)


In [95]:
x_max = np.max(trans_locs[:,0])
y_max = np.max(trans_locs[:,1])
z_max = np.max(trans_locs[:,2])

platonic_cone_arr = np.zeros((x_max, y_max, z_max))
print(platonic_cone_arr.shape)
print(np.asarray(np.where(platonic_cone_arr==255)))
# for [locx,locy,locz] in trans_locs[:,:]:
for i in np.arange(trans_locs.shape[0]):
    [locx,locy,locz] = trans_locs[i,:].astype(int)-1
    platonic_cone_arr[locx,locy,locz]=255
print(np.asarray(np.where(platonic_cone_arr==255)).shape)


(49, 31, 22)
[]
(3, 16111)


In [96]:
cone_f = h5py.File('./DONE_HDFS/ygw18/platonic_cone.h5', 'w')
d = cone_f.create_dataset('cone', data=platonic_cone_arr)
cone_f.close()